In [1]:
!pip3 install mesa

     |████████████████████████████████| 668 kB 19.2 MB/s 
     |████████████████████████████████| 60 kB 4.1 MB/s 


In [2]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `Multigrid` que nos deja tener varios agentes en una celda
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation
#from mesa.time import RandomActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

import signal

import math

In [3]:
class Move(Agent):
    #Inicializacion
    grid = None
    x = None
    y = None
    moore = False
    include_center=False

    def __init__(self, unique_id, pos, model, moore=False): #moore es para moverse en 8 direcciones
    
        super().__init__(unique_id, model)
        self.pos = pos
        self.moore = moore
    
    def move(self):
        next_moves = self.model.grid.get_neighborhood(self.pos,moore=False,include_center=False)
        posicion_azar=random.choice(next_moves)
        if self.model.grid.is_cell_empty(posicion_azar):
            self.movimientos += 1

In [4]:
Lights=[]
light_state=[]
carCounter = 0
CarsA = []
CarsB = []
CarsC = []
CarsD = []
gen = 0
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for agent in cell_content:
            grid[x][y] = agent.height
    return grid

class UC(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.contador=0
    self.height=0
    self.step_count=0
    self.inBetween = 0
    self.isStart = True

  def allEmpty(self):
    global carCounter
    carCounter = 0
    for i in range(4):
      if light_state[i].check_car():
        carCounter+=1
    if carCounter == 0:
      carCounter = 0
      return True
    return False

  def oneYellow(self):
    global carCounter
    carCounter = 0
    index = ''
    for i in range(4):
      if light_state[i].check_car():
        carCounter+=1
        index = i
    if carCounter == 1:
      carCounter = 0
      return index
    return -1

  def setAllRed(self):
    light_state[self.contador].state=6
    light_state[self.contador-1].state=6
    light_state[self.contador-2].state=6
    light_state[self.contador-3].state=6

  def change_light(self):
    if self.allEmpty():
        light_state[self.contador].state=7
        light_state[self.contador-1].state=7
        light_state[self.contador-2].state=7
        light_state[self.contador-3].state=7
        return
    yellowIndex = self.oneYellow()
    if yellowIndex !=-1:
      print('entro yellow')
      light_state[yellowIndex].state=8
      light_state[yellowIndex-1].state=6
      light_state[yellowIndex-2].state=6
      light_state[yellowIndex-3].state=6
      return 
    if self.contador == 3:
      if light_state[self.contador].check_car():
        light_state[self.contador-1].state=6
        light_state[self.contador-2].state=6
        light_state[self.contador-3].state=6
        light_state[self.contador].state=8
        self.contador = 0
      else:
        self.contador = 0
        self.change_light()
    else:
      if light_state[self.contador].check_car():
        light_state[self.contador-1].state=6
        light_state[self.contador-2].state=6
        light_state[self.contador-3].state=6
        light_state[self.contador].state=8
        self.contador +=1
      else: 
        self.contador+=1
        self.change_light()

  def step(self):
    if self.step_count == 5:
      self.inBetween +=1
    else: self.step_count+=1

  def advance(self):
    if self.isStart:
      self.change_light()
      self.isStart = False
    if self.step_count==5:
      self.setAllRed()
    if self.inBetween > 6:
      self.change_light()
      self.step_count=0
      self.inBetween = 0

class Light(Agent):
  #global Lights=list(((8,5),(8,8),(5,8),(5,5)))
  def __init__(self, unique_id, model, direction):
    super().__init__(unique_id, model)
    self.state=7 #rojo = 6, amarillo=7, verde =8
    self.height=self.state
    self.direction = direction
    
  def check_car(self):
    neighbours = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False,)
    for neighbor in neighbours:
      if isinstance(neighbor, Car) and neighbor.direction == self.direction:
        return True 
    return False

  def step(self):
    self.check_car()
    self.height=self.state

class Car(Agent):
  def __init__(self, unique_id, model, direction, vuelta, height):
    super().__init__(unique_id, model)
    self.height= height
    self.vuelta= vuelta #0 = izquierda, 1=recto, 2=derecha
    self.is_moving=True 
    self.direction = direction
    self.once_light=False #ver si ya paso un semaforo

  def canIMoveThere(self, pos):
    neighbours = self.model.grid.get_neighbors(
        self.pos,
        moore=False,
        include_center=False)
    for neighbour in neighbours:
      if (isinstance(neighbour, Car) and neighbour.pos == pos and neighbour.height > 1):
        return False
    return True   

  def checkBoundaries(self,pos, posNext):
    x, y = pos
    xNext, yNext = posNext
    if (x == 0 and xNext == -1):
      self.height = 1
      return False
    if (y == 0 and yNext == -1):
      self.height = 1
      return False
    if (x == self.model.height - 1 and xNext == self.model.height):
      self.height = 1
      return False
    if (y == self.model.width - 1 and yNext == self.model.width):
      self.height = 1
      return False
    return True
  
    '''
    0-A, 0-B, 0-C, 0-D
    
    2-A, 2-B, 2-C, 2-D
    '''
  def turnLeft(self):
    x, y = self.pos

    if (self.direction == 'A'):
      if (y != 7):
        if self.checkBoundaries(self.pos, (x,y+1)) and self.canIMoveThere((x,y+1)):
          self.model.grid.move_agent(self, (x,y+1))
      elif self.checkBoundaries(self.pos, (x-1,y)) and self.canIMoveThere((x-1,y)) :
        self.model.grid.move_agent(self, (x-1,y))
        
    if (self.direction == 'B'):
      if (x != 6):
        if self.checkBoundaries(self.pos, (x-1,y)) and self.canIMoveThere((x-1,y)):
          self.model.grid.move_agent(self, (x-1,y))
      elif self.checkBoundaries(self.pos, (x,y-1)) and self.canIMoveThere((x,y-1)):
        self.model.grid.move_agent(self, (x,y-1))

    if (self.direction == 'C'):
      if (y != 6):
        if self.checkBoundaries(self.pos, (x,y-1)) and self.canIMoveThere((x,y-1)):
          self.model.grid.move_agent(self, (x,y-1))
      elif self.checkBoundaries(self.pos, (x+1,y)) and self.canIMoveThere((x+1,y)):
        self.model.grid.move_agent(self, (x+1,y))

    if (self.direction == 'D'):
      if (x != 7):
        if self.checkBoundaries(self.pos, (x+1,y)) and self.canIMoveThere((x+1,y)):
          self.model.grid.move_agent(self, (x+1,y))
      elif self.checkBoundaries(self.pos, (x,y+1)) and self.canIMoveThere((x,y+1)):
        self.model.grid.move_agent(self, (x,y+1))

  def turnRight(self):
    x, y = self.pos

    if (self.direction == 'A'):
      if (y != 5):
        if self.checkBoundaries(self.pos, (x,y+1)) and self.canIMoveThere((x,y+1)):
          self.model.grid.move_agent(self, (x,y+1))
      elif self.checkBoundaries(self.pos, (x+1,y)) and self.canIMoveThere((x+1,y)):
        self.model.grid.move_agent(self, (x+1,y))
        
    if (self.direction == 'B'):
      if (x != 8):
        if self.checkBoundaries(self.pos, (x-1,y)) and self.canIMoveThere((x-1,y)):
          self.model.grid.move_agent(self, (x-1,y))
      elif self.checkBoundaries(self.pos, (x,y+1)) and self.canIMoveThere((x,y+1)):
        self.model.grid.move_agent(self, (x,y+1))

    if (self.direction == 'C'):
      if (y != 8):
        if self.checkBoundaries(self.pos, (x,y-1)) and self.canIMoveThere((x,y-1)):
          self.model.grid.move_agent(self, (x,y-1))
      elif self.checkBoundaries(self.pos, (x-1,y)) and self.canIMoveThere((x-1,y)):
        self.model.grid.move_agent(self, (x-1,y))

    if (self.direction == 'D'):
      if (x != 5):
        if self.checkBoundaries(self.pos, (x+1,y)) and self.canIMoveThere((x+1,y)):
          self.model.grid.move_agent(self, (x+1,y))
      elif self.checkBoundaries(self.pos, (x,y-1)) and self.canIMoveThere((x,y-1)):
        self.model.grid.move_agent(self, (x,y-1))

  def turnStraight(self):
    x, y = self.pos
    if (self.direction == 'A'):
      if self.checkBoundaries(self.pos, (x,y+1)) and self.canIMoveThere((x,y+1)):
        self.model.grid.move_agent(self, (x,y+1))
    if (self.direction == 'B'):
      if self.checkBoundaries(self.pos, (x-1,y)) and self.canIMoveThere((x-1,y)):
        self.model.grid.move_agent(self, (x-1,y))
    if (self.direction == 'C'):
      if self.checkBoundaries(self.pos, (x,y-1)) and self.canIMoveThere((x,y-1)):
        self.model.grid.move_agent(self, (x,y-1))
    if (self.direction == 'D'):
      if self.checkBoundaries(self.pos, (x+1,y)) and self.canIMoveThere((x+1,y)):
        self.model.grid.move_agent(self, (x+1,y))          


  def brakeLight(self):
    global carCounter
    neighbours = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False,)
    for neighbor in neighbours:
      if isinstance(neighbor, Light) and neighbor.direction == self.direction and self.once_light==False:
        if neighbor.state == 7 and carCounter == 1: # Yellow Light
          self.is_moving = False
        elif neighbor.state !=8: # Green light
          self.is_moving = False  
        else: # Red Light
          self.is_moving = True
          self.once_light=True

  def move(self):
    if self.vuelta == 0:
      self.turnLeft()         
    if self.vuelta == 1:
      self.turnStraight()          
    if self.vuelta == 2:
      self.turnRight()

  def step(self):
    self.brakeLight()

  def advance(self):
    if self.is_moving: 
      self.move()

class Street(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.height = 1
      

class IntersectionModel(Model):
  def __init__(self, width, height):
    self.grid = MultiGrid(width, height, True)
    self.schedule = SimultaneousActivation(self)
    self.width = width
    self.height = height
    global CarsA, CarsB, CarsC, CarsD, gen
    CarsA=list(((7,0),(8,0)))
    CarsB=list(((13,7),(13,8)))
    CarsC=list(((5,13),(6,13)))
    CarsD=list(((0,5),(0,6)))
    Lights=list(((8,5),(8,8),(5,8),(5,5)))

    Directions=list(('A','B','C','D'))
    #Street
    StreetPos = [
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,0,0,0,0,0]
    ]
    for (content, x, y) in self.grid.coord_iter():
      if StreetPos[x][y] == 1:
        w = Street((x, y), self)
        self.grid.place_agent(w, (x,y)) 

    #semaforos
    for z in range(4):
      l = Light(Lights[z], self, Directions[z])
      global light_state
      light_state.append(l)
      self.grid.place_agent(l, Lights[z]) 
      self.schedule.add(l)

    # Aquí definimos con colector para obtener el grid completo.
    self.datacollector = DataCollector(model_reporters={"Grid": get_grid})
    #UC
    uc=UC(self,(0,0))
    self.grid.place_agent(uc, (0,0))   
    self.schedule.add(uc)


  def placeCarA(self):
    global CarsA, gen
    gen+=1
    z = np.random.choice([0,1])
    x, y = CarsA[z]
    if (x == 7):
      vuelta = np.random.choice([0,1])
    elif (x == 8):
      vuelta = np.random.choice([1,2])
    c = Car((CarsA[z], gen), self, 'A', vuelta, 2)
    self.grid.place_agent(c, CarsA[z])   
    self.schedule.add(c)

  def placeCarB(self):
    global CarsB, gen
    gen+=1
    z = np.random.choice([0,1])
    x, y = CarsB[z]
    if (y == 7):
      vuelta = np.random.choice([0,1])
    elif (y == 8):
      vuelta = np.random.choice([1,2])
    c = Car((CarsB[z],gen), self, 'B', vuelta, 3)
    self.grid.place_agent(c, CarsB[z]) 
    self.schedule.add(c)

  def placeCarC(self):
    global CarsC, gen
    gen+=1
    z = np.random.choice([0,1])
    x, y = CarsC[z]
    if (x == 6):
      vuelta = np.random.choice([0,1])
    elif (x == 5):
      vuelta = np.random.choice([1,2])
    c = Car((CarsC[z], gen), self, 'C', vuelta, 4)
    self.grid.place_agent(c, CarsC[z]) 
    self.schedule.add(c)

  
  def placeCarD(self):
    global CarsD, gen
    gen+=1
    z = np.random.choice([0,1])
    x, y = CarsD[z]
    if (y == 6):
      vuelta = np.random.choice([0,1])
    elif (y == 5):
      vuelta = np.random.choice([1,2])
    c = Car((CarsD[z], gen), self, 'D', vuelta, 5)
    self.grid.place_agent(c, CarsD[z]) 
    self.schedule.add(c)
  

  def agentPlacer(self):
    #Cars
    global CarsA, CarsB, CarsC, CarsD
    randomList = np.random.randint(1,4)
    if randomList == 1: self.placeCarA()
    if randomList == 2: self.placeCarB()
    if randomList == 3: self.placeCarC()
    if randomList == 4: self.placeCarD()



  def step(self):
    placeCar = np.random.randint(0,5)
    if placeCar == 3:
      self.agentPlacer()
    self.datacollector.collect(self)
    self.schedule.step()

In [5]:
# Definimos el tamaño del Grid
GRID_SIZE = 14


# Max RunTime
start_time = time.time()
maxTime = .1

frameCounter = 0

model = IntersectionModel(GRID_SIZE, GRID_SIZE)
while(time.time() - start_time) < maxTime:
  frameCounter+=1
  model.step()

frameCounter+=1
model.step()


# Tiempo de Ejecucion
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))


# Numero de Movimientos Realizados por los Agentes
print('Numero de Movimientos Realizados: ', frameCounter)


entro yellow
entro yellow
Tiempo de ejecución: 0:00:00.101302
Numero de Movimientos Realizados:  195


In [6]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [7]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.Greens)
plt.colorbar()

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=frameCounter)

In [8]:
anim